<a href="https://colab.research.google.com/github/sfwtopoulos/stocks/blob/master/Stocks_seq_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [91]:
pip install -q talos

     |████████████████████████████████| 26.6MB 165kB/s 
ERROR: umap-learn 0.4.2 has requirement scipy>=1.3.1, but you'll have scipy 1.2.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


In [0]:
pip install -q wget

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import math
import warnings
import numpy as np
import time
import sys

import pandas_datareader as pdr
#%tensorflow_version 2.x
# Restart runtime using 'Runtime' -> 'Restart runtime...'
%tensorflow_version 1.x
import tensorflow as tf
import keras


# from keras.layers import LSTM
# from keras.models import Sequential
# from keras.layers.wrappers import TimeDistributed
# from keras.layers.wrappers import TimeDistributed
# from keras.layers.core import Dense, Activation, Dropout
# from keras.preprocessing.sequence import pad_sequences

from tensorflow.python.keras.layers import Dense, Activation, Dropout, Input
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import LSTM
from tensorflow.python.keras.layers.wrappers import TimeDistributed
from tensorflow.python.keras.layers.wrappers import TimeDistributed
from tensorflow.python.keras.preprocessing.sequence import pad_sequences


from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt

import os
import wget
from zipfile import ZipFile

warnings.filterwarnings('ignore',category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [0]:
# from keras.optimizers import Adam, Nadam
# from keras.activations import softmax
# from keras.losses import categorical_crossentropy, logcosh
# from talos.utils import lr_normalizer
import talos
from tensorflow.python.keras.optimizers import Adam, Nadam
from tensorflow.python.keras.activations import softmax
from tensorflow.python.keras.losses import categorical_crossentropy, logcosh
from talos.utils import lr_normalizer

In [95]:
print(keras.__version__)
print(tf.__version__)
#print(tensorflow.compat.v2.__version__)

2.3.0
1.15.2


In [109]:
#Flag to select the small or the large dataset from the git repo
large=True;

if not large:
  #Small Dataset
  #Data import from git repo
  print("Small Dataset Loading")
  url = 'https://raw.githubusercontent.com/sfwtopoulos/stocks/master/stocks_dataset/combined.csv'
  #df1 = pd.read_csv(url, error_bad_lines=False)
  dfstocks = pd.read_csv(url, sep=',')
  dfstocks=dfstocks.rename({' AAPL':'Company', ' Close/Last':'Close', ' Volume':'Volume', ' Open':'Open', ' High':'High', ' Low':'Low'}, axis=1);
else:
  #Large Dataset
  print("Large Dataset Loading")
  url = 'https://raw.githubusercontent.com/sfwtopoulos/stocks/master/stocks_dataset/combined_large.zip'
  wget.download(url, '.')
  # Create a ZipFile Object and load sample.zip in it
  with ZipFile('combined_large.zip', 'r') as zipObj:
    # Extract all the contents of zip file in current directory
    zipObj.extractall()
  print(os.listdir("."))
  dfstocks = pd.read_csv('combined_large.csv', sep=',') 
  dfstocks=dfstocks[~dfstocks.Date.str.contains("nan")]
  dfstocks=dfstocks[~dfstocks.Date.str.contains("Low")]
  dfstocks=dfstocks[~dfstocks.Date.str.contains("N/A")]
  dfstocks=dfstocks[~dfstocks.Date.str.contains("date")]
  dfstocks.Date=pd.to_datetime(dfstocks.Date)
  # dfstocks['Date'] = dfstocks['Date'].dt.strftime('%Y-%m-%d')
  dfstocks['Date'] = dfstocks['Date'].dt.strftime('%d-%m-%Y')
  dfstocks=dfstocks.rename({' AA':'Company', ' Close/Last':'Close', ' Volume':'Volume', ' Open':'Open', ' High':'High', ' Low':'Low'}, axis=1);

Large Dataset Loading
['.config', 'combined_large.csv', 'combined_large (1).zip', 'sub_split_large.csv', 'gdrive', 'combined_large.zip', 'sample_data']


In [111]:
type(dfstocks['Date'])
dfstocks.dtypes

Date       object
Close      object
Volume     object
Open       object
High       object
Low        object
Company    object
dtype: object

In [112]:
for col in dfstocks.columns: 
    print(col) 

Date
Close
Volume
Open
High
Low
Company


In [113]:
dfstocks

,Date,Close,Volume,Open,High,Low,Company
0,11-02-2018,36.93000031,2681700,37,37.68999863,36.40999985,AA
1,11-01-2018,36.77999878,2756200,35.43999863,36.88000107,35.24000168,AA
2,31-10-2018,34.99000168,3380700,34.81999969,35.82099915,34.59999847,AA
3,30-10-2018,34.15000153,2193400,33.65000153,34.47999954,33.4129982,AA
4,29-10-2018,33.81999969,2219600,34.59999847,35.11999893,33.40000153,AA
...,...,...,...,...,...,...,...
304988,05-09-1973,0.469444454,79200,0.469444454,0.48888889,0.469444454,USB
304989,05-08-1973,0.469444454,90000,0.469444454,0.480555564,0.469444454,USB
304990,05-07-1973,0.461111099,43200,0.469444454,0.480555564,0.461111099,USB
304991,05-04-1973,0.469444454,10800,0.469444454,0.480555564,0.469444454,USB


In [0]:
#drop $sign from values
dfstocks.Close=dfstocks['Close'].astype(str)
dfstocks.Close=dfstocks.Close.apply(lambda x: x.replace('$',''))
dfstocks.Open=dfstocks['Open'].astype(str)
dfstocks.Open=dfstocks.Open.apply(lambda x: x.replace('$',''))
dfstocks.High=dfstocks['High'].astype(str)
dfstocks.High=dfstocks.High.apply(lambda x: x.replace('$',''))
dfstocks.Low=dfstocks['Low'].astype(str)
dfstocks.Low=dfstocks.Low.apply(lambda x: x.replace('$',''))

In [115]:
#drop rows containing nan or header from the csv files
dfstocks=dfstocks[~dfstocks.Low.str.contains("nan")]
dfstocks=dfstocks[~dfstocks.Low.str.contains("Low")]
dfstocks=dfstocks[~dfstocks.Low.str.contains("N/A")]
dfstocks=dfstocks[~dfstocks.Volume.str.contains("N/A")]
dfstocks.reset_index(drop=True, inplace=True)
dfstocks.Date.count()

304968

In [116]:
#sort dataframe based on date and Company Name
dfstocks = dfstocks.sort_values(['Date', 'Company'])
dfstocks.head(100)

,Date,Close,Volume,Open,High,Low,Company
12322,01-02-1970,7.140914917,22500,7.140914917,7.225019932,7.140914917,AA
34201,01-02-1970,6.699775219,24100,6.699775219,6.770052433,6.699775219,ARNC
81427,01-02-1970,11.16975021,24400,11.0994997,11.20487499,10.99412537,CNP
93750,01-02-1970,0.683144331,1109700,0.688280761,0.689564824,0.683144331,DIS
141129,01-02-1970,0.798177063,2227200,0.807291687,0.80859375,0.796875,GE
...,...,...,...,...,...,...,...
212939,01-02-1976,1.448489547,21200,1.448489547,1.498437524,1.448489547,MATX
224420,01-02-1976,1.114583373,1108800,1.104166627,1.114583373,1.104166627,MO
236743,01-02-1976,1.934027791,680400,1.923611164,1.9375,1.916666627,MRK
249066,01-02-1976,10.83227444,81100,10.77013874,10.83227444,10.77013874,MRO


In [117]:
#Converting Datatypes
dfstocks.Date=pd.to_datetime(dfstocks.Date)
dfstocks.Close=pd.to_numeric(dfstocks.Close)
dfstocks.Volume=pd.to_numeric(dfstocks.Volume)
dfstocks.Open=pd.to_numeric(dfstocks.Open)
dfstocks.High=pd.to_numeric(dfstocks.High)
dfstocks.Low=pd.to_numeric(dfstocks.Low)
dfstocks.dtypes

Date       datetime64[ns]
Close             float64
Volume              int64
Open              float64
High              float64
Low               float64
Company            object
dtype: object

In [0]:
#Company Chosen
comp="AAPL"

In [119]:
#dfstocks[dfstocks['Company'].str.contains("AMZN")]
sub_split=dfstocks[dfstocks['Company'].str.contains(comp)]
sub_split.isnull().values.any()
sub_split.isna().values.any()
sub_split = sub_split.sort_values(['Date'])
sub_split.reset_index(drop=True, inplace=True)
sub_split.shape[0]

9556

In [0]:
#Run only if you want to have a subset hidden from talos
#hidden split for validation
hidden_split=sub_split[(sub_split['Date'].dt.year==2018) | (sub_split['Date'].dt.year==2017) | (sub_split['Date'].dt.year==2016)]

#drop hidden_split from sub_split 
sub_split=sub_split[sub_split['Date'].dt.year < 2016]
hidden_split

In [120]:
sub_split

,Date,Close,Volume,Open,High,Low,Company
0,1980-12-12,0.513393,117258400,0.513393,0.515625,0.513393,AAPL
1,1980-12-15,0.486607,43971200,0.488839,0.488839,0.486607,AAPL
2,1980-12-16,0.450893,26432000,0.453125,0.453125,0.450893,AAPL
3,1980-12-17,0.462054,21610400,0.462054,0.464286,0.462054,AAPL
4,1980-12-18,0.475446,18362400,0.475446,0.477679,0.475446,AAPL
...,...,...,...,...,...,...,...
9551,2018-10-29,212.240006,45935500,219.190002,219.690002,206.089996,AAPL
9552,2018-10-30,213.300003,36660000,211.149994,215.179993,209.270004,AAPL
9553,2018-10-31,218.860001,38358900,216.880005,220.449997,216.619995,AAPL
9554,2018-11-01,222.220001,58323200,219.050003,222.360001,216.809998,AAPL


In [0]:
# prepare training and testing data sets for LSTM based sequence modeling
#def dimitris_function(time_series, scaling=True,split_ratio=0.9):
def dimitris_function(time_series,split_ratio=0.9):

    series1 = pd.Series(time_series[time_series.columns[0]])
    series2 = pd.Series(time_series[time_series.columns[1]])
    series3 = pd.Series(time_series[time_series.columns[2]])
    series4 = pd.Series(time_series[time_series.columns[3]])
    series5 = pd.Series(time_series[time_series.columns[4]])

    values1 = series1.values
    values1 = values1.reshape((len(values1), 1))
    values2 = series2.values
    values2 = values2.reshape((len(values2), 1))
    values3 = series3.values
    values3 = values3.reshape((len(values3), 1))
    values4 = series4.values
    values4 = values4.reshape((len(values4), 1))
    values5 = series5.values
    values5 = values5.reshape((len(values5), 1))

    #scaler2 --> Close Price Scaler
    scaler1 = MinMaxScaler(feature_range=(0, 1))
    time_series["scaled1"] = scaler1.fit_transform(values1)
    scaler2 = MinMaxScaler(feature_range=(0, 1))
    time_series["scaled2"]= scaler2.fit_transform(values2)
    scaler3 = MinMaxScaler(feature_range=(0, 1))
    time_series["scaled3"]= scaler3.fit_transform(values3)
    scaler4 = MinMaxScaler(feature_range=(0, 1))
    time_series["scaled4"]= scaler4.fit_transform(values4)
    scaler5 = MinMaxScaler(feature_range=(0, 1))
    time_series["scaled5"]= scaler5.fit_transform(values5)


    time_series = time_series[["scaled1","scaled2","scaled3","scaled4","scaled5"]].values

        
    split_ratio = int(len(time_series) * split_ratio)

    train = time_series[0:split_ratio]
    test = time_series[split_ratio:len(time_series)-1]
    
    # return train,test,scaler1,scaler2
    return train,test,scaler2


In [22]:
#mounting gdrive to save talos results for each run and tensor flow checkpoints
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#def get_seq_model(hidden_units1=40, hidden_units2=65, hidden_units3=32, drop=0.2,input_shape=(1,1)):
def get_seq_model(x_train, y_train, x_val, y_val, params):  

    # create and fit the LSTM network
    model = Sequential()

    print(params)
    print(type(params))

    # samples*timesteps*features
    # https://keras.io/getting-started/sequential-model-guide/
    #https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0227222

    ######!!!!!#####!!!!!
    #(train_x.shape[1],5) where 5 is the number of features is equal to :
    #(train_x.shape[1],train_x.shape[2])
    model.add(LSTM(
                   input_shape=(x_train.shape[1],x_train.shape[2]),
                   units = params['hidden_1'],  
                   #activation='relu',recurrent_activation='sigmoid',
                   return_sequences=True
    ))
    model.add(Dropout(params['dropout']))
    model.add(LSTM(
                   units = params['hidden_2'],
                   #activation='relu',recurrent_activation='sigmoid',
                   return_sequences=True
    ))
    model.add(Dropout(params['dropout']))   
    model.add(LSTM(
                   units = params['hidden_3'], 
                   #activation='relu',recurrent_activation='sigmoid',
                   return_sequences=True
    ))
    model.add(Dropout(params['dropout']))

    # readout layer. TimeDistributedDense uses the same weights for all
    # time steps.
    # model.add(TimeDistributed(Dense(1))) #number of inputs in the 1st layer
   # model.add(TimeDistributed(Dense(5)))
    model.add(TimeDistributed(Dense(5)))
    start = time.time()
    
    model.compile(loss="mse", optimizer="rmsprop")
    timestr = time.strftime("%Y%m%d-%H%M%S")
    checkpoint_path = "/content/gdrive/My Drive/checkpoints1"  + "/cp-{epoch:04d}.ckpt"
    checkpoint_dir = os.path.dirname(checkpoint_path)
    cp_callback = tf.keras.callbacks.ModelCheckpoint(
        checkpoint_path, verbose=1, save_weights_only=True,#save_best_only='TRUE'
        # Save weights, every 5-epochs.
        period=15)
    
   # early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=40)
    
    out = model.fit(x_train, y_train,
              epochs=params['epochs'], batch_size=1,
              validation_data=[x_val, y_val],
              #epochs=200, batch_size=1,
              verbose=1,callbacks=[cp_callback])

    return out, model

In [24]:
ls gdrive/"My Drive"/checkpoints1

checkpoint                        cp-0060.ckpt.data-00000-of-00002
cp-0015.ckpt.data-00000-of-00002  cp-0060.ckpt.data-00001-of-00002
cp-0015.ckpt.data-00001-of-00002  cp-0060.ckpt.index
cp-0015.ckpt.index                cp-0075.ckpt.data-00000-of-00002
cp-0030.ckpt.data-00000-of-00002  cp-0075.ckpt.data-00001-of-00002
cp-0030.ckpt.data-00001-of-00002  cp-0075.ckpt.index
cp-0030.ckpt.index                cp-0090.ckpt.data-00000-of-00002
cp-0045.ckpt.data-00000-of-00002  cp-0090.ckpt.data-00001-of-00002
cp-0045.ckpt.data-00001-of-00002  cp-0090.ckpt.index
cp-0045.ckpt.index


In [0]:
talos=False;

if not talos:
  #Chosen final parameters
  p = {'hidden_1': 8,
     'hidden_2': 16,
     'hidden_3': 32,
     'dropout': 0.1,
     'epochs': 100
     }

else:
  #Parameters for the model optimization in talos
  p = {'hidden_1':[8, 16, 32],
     'hidden_2':[8, 16, 32],
     'hidden_3':[8, 16, 32],
     'dropout': [0.1, 0.2],
     #'dropout': [0.1],
     'epochs': [20, 50, 100]
     }

In [0]:
# print(type(sub_split.Close))
sub_split_date=sub_split[['Date']]
sub_split=sub_split[['Volume', 'Close', 'Open', 'High', 'Low']]
#sub_split.columns[1]

In [125]:
# split train and test datasets
train,test,scaler_Close = dimitris_function(sub_split,split_ratio=0.7)

#train
print(len(train))
print(train.shape[0])
print(train.shape)
print(test.shape)
# print(train)

6689
6689
(6689, 5)
(2866, 5)


In [126]:
#We use numpy to reshape our time series into 3D tensors.
train = np.reshape(train,(train.shape[0],1,5))
test = np.reshape(test,(test.shape[0],1,5))

train_x = train[:,:,:]
train_y = train[:,:,:]

test_x = test[:,:,:]
test_y = test[:,:,:]

print("Data Split Complete")

print("train_x shape={}".format(train_x.shape))
print("train_y shape={}".format(train_y.shape))
print("test_x shape={}".format(test_x.shape))
print("test_y shape={}".format(test_y.shape))

Data Split Complete
train_x shape=(6689, 1, 5)
train_y shape=(6689, 1, 5)
test_x shape=(2866, 1, 5)
test_y shape=(2866, 1, 5)


In [127]:
load_from_checkpoint=True;

seq_lstm_model=None
if load_from_checkpoint:
  #Load the latest checkpoint saved in Drive #'gdrive/My Drive/checkpoints1/cp-0090.ckpt'
  latest=tf.train.latest_checkpoint("gdrive/My Drive/checkpoints1")
  print(latest)
  #Create the model and train for 1 epoch to get the NN stracture
  p['epochs']=1
  try:
    seq_lstm_model=get_seq_model(train_x,train_y,test_x,test_y,p)
  except:
    print("Model Build Failed. Trying Again")
    seq_lstm_model = get_seq_model(train_x,train_y,test_x,test_y,p)
  #Load the weight of the checkpoint to the NN 
  seq_lstm_model[1].load_weights(latest)
else:
  # build RNN model
  try:
    seq_lstm_model = get_seq_model(train_x,train_y,test_x,test_y,p)
  except:
    print("Model Build Failed. Trying Again")
    seq_lstm_model = get_seq_model(train_x,train_y,test_x,test_y,p)

gdrive/My Drive/checkpoints1/cp-0090.ckpt
{'hidden_1': 8, 'hidden_2': 16, 'hidden_3': 32, 'dropout': 0.1, 'epochs': 1}
<class 'dict'>
Train on 6689 samples, validate on 2866 samples
6689/6689 [==============================] - 46s 7ms/sample - loss: 1.1897e-04 - val_loss: 0.0119


In [0]:
# x=np.concatenate((train_x, test_x), axis=1)
# y=np.concatenate((train_y, test_y), axis=1)
# print("x shape={}".format(x.shape))
# print("y shape={}".format(y.shape))
import talos


In [0]:
%%capture
# 100	0.0005776109765	0.001970537574	0.1	100	8	16	32
scan_obj = talos.Scan(x=train_x,
                      y=train_y,
                      x_val=test_x,
                      #x_val=train_x,
                      y_val=test_y,
                     #y_val=train_y,

                      params=p,
                      val_split=0.1,
                      model=get_seq_model,
                      #shuffle=False,
                      experiment_name='gdrive/My Drive/test',
                      #fraction_limit=0.1
                      )


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
print("Finished Talos Scan!!")

In [0]:
scan_obj.data.head()

In [0]:
# train_y

In [0]:
# train the model
#https://medium.com/@rajath95/how-to-save-and-upload-deep-learning-machine-learning-models-in-google-colab-using-google-drive-eb2901937ae
# seq_lstm_model[1].fit(train_x, train_y,
#                 epochs=2, batch_size=1,
#                 #epochs=200, batch_size=1,
#                 verbose=1)
# print("Model Fit Complete")

In [128]:
# train fit performance
trainPredict = seq_lstm_model[1].predict(train_x)
trainScore = math.sqrt(mean_squared_error(train_y[0], trainPredict[0]))
print('Train Score: %.2f RMSE' % (trainScore))

# Pad input sequence
testPredict = pad_sequences(test_x,maxlen=train_x.shape[1],padding='post',dtype='float64')

# forecast values
testPredict = seq_lstm_model[1].predict(testPredict)
testPredict

Train Score: 0.04 RMSE


array([[[0.00681369, 0.07496563, 0.07447192, 0.07334421, 0.07643311]],

       [[0.00677703, 0.07424965, 0.07376415, 0.07264255, 0.07571185]],

       [[0.0067693 , 0.07401095, 0.07352845, 0.0724088 , 0.07547195]],

       ...,

       [[0.01491153, 0.76234865, 0.7542589 , 0.7511917 , 0.7646524 ]],

       [[0.01496051, 0.7703818 , 0.7621977 , 0.75911766, 0.7726877 ]],

       [[0.01501237, 0.77499855, 0.76675797, 0.7636628 , 0.7773174 ]]],
      dtype=float32)

In [0]:
# print()

In [0]:
# trainPredict

In [0]:
# # forecast values
# testPredict = seq_lstm_model[1].predict(testPredict)
# testPredict

In [0]:
#Drop all other variables except Close Price Output
testPredict=np.delete(testPredict, [2,3,4], 2)
trainPredict=np.delete(trainPredict, [2,3,4], 2)

testPredict=np.delete(testPredict, 0, 2)
trainPredict=np.delete(trainPredict, 0, 2)

# # inverse transformation of the Close Price
trainPredict = scaler_Close.inverse_transform(trainPredict.\
                                        reshape(-1,1))
testPredict = scaler_Close.inverse_transform(testPredict.\
                                       reshape(-1,1))

In [0]:
# testPredict

In [0]:
# trainPredict

In [0]:
# # # inverse transformation of the Close Price
# trainPredict = scaler_Close.inverse_transform(trainPredict.\
#                                         reshape(-1,1))
# testPredict = scaler_Close.inverse_transform(testPredict.\
#                                        reshape(-1,1))

In [0]:
# trainPredict

In [133]:
# plot the true and forecasted values
split_ratio = len(trainPredict)+1

plt.plot(sub_split_date.Date,
          sub_split.Close.values,c='black',
          alpha=0.9,label='True Data')

plt.plot(sub_split_date.Date[1:split_ratio],
          trainPredict,label='Training Fit',c='g')

# plt.plot(sub_split_date.Date[split_ratio+1:],
#           testPredict[:test_x.shape[1]],label='Testing Forecast')

plt.plot(sub_split_date.Date[split_ratio:],
          testPredict[:test_x.shape[0]],label='Testing Forecast')

plt.title('Forecast Plot')
plt.legend()
#plt.show()
plt.rcParams["figure.figsize"] = (40,20)
plt.show()